In [1]:
import pandas as pd
import glob

#Scraping
import requests
from bs4 import BeautifulSoup as bs

#working on scraped Data
import re

#Writing
#import xlsxwriter
import numpy as np

In [ ]:
#Set folder from which to retrieve all CSVs
all_files = glob.glob("/Users/FelixHoffmann/Desktop/GreenLeaders/Hotellists"+"/*.csv")

#Create empty DataFrame
full_frame=pd.DataFrame()

#for each file
for counter, f in enumerate(all_files):
    #read the csv
    df=pd.read_csv(f)
    #add column with country name
    df['Country']=all_files[counter][60:-4]
    #add two-column DataFrame to final DataFrame
    full_frame=pd.concat([full_frame,df])

In [ ]:
HotelListIndexing=0

for m in range(270):
    counter = 0
    final_list=[]
    
    while counter<1000:
        
        r=requests.get(full_frame.iloc[HotelListIndexing+counter]['link'])
        s=bs(r.content, 'lxml')

        #Name
        HotelName = str(s.h1.string)

        #Average TA user Rating
        AvgRating = float(s.find_all('span', {'class':'_3cjYfwwQ'})[0].text.strip()) if len(s.find_all('span', {'class':'_3cjYfwwQ'}))>0 else 'null'

        #Awards
        awards=s.find_all('div', {'class':'GzEo7hAU'})
        if len(awards) == 2:
            TravelersChoice = awards[0].text 
            GreenLeader     = awards[1].text
        elif len(awards) == 1: 
            if "Travel" in awards[0].text:
                TravelersChoice = awards[0].text 
                GreenLeader     = 'null'      
            else: 
                GreenLeader     = awards[0].text 
                TravelersChoice = 'null'        
        else:
            TravelersChoice = 'null'
            GreenLeader     = 'null'

        #Total Photos
        TotalPhotos=int(s.find_all('span',{'class':'is-hidden-tablet DzxX_Y73'})[0].text.replace(",","")) if len(s.find_all('span',{'class':'is-hidden-tablet DzxX_Y73'}))>0 else 0

        #Picture Subgroups
        DiningPhotos = PoolBeachPhotos = RoomSuitePhotos = TravelerPhotos = VideosPhotos  = 'null'
        for i in range(len(s.find_all('div', {'class':'Xjl42ubC FiqYmbeD'}))):
            l=s.find_all('div', {'class':'Xjl42ubC FiqYmbeD'})[i].text.strip()
            if l[:6] == 'Dining': DiningPhotos    = int(re.findall(r'\d+', l)[0].replace(",",""))
            if l[:6] == 'Pool &': PoolBeachPhotos = int(re.findall(r'\d+', l)[0].replace(",",""))
            if l[:6] == 'Room &': RoomSuitePhotos = int(re.findall(r'\d+', l)[0].replace(",",""))
            if l[:6] == 'Travel': TravelerPhotos  = int(re.findall(r'\d+', l)[0].replace(",",""))
            if l[:6] == 'Videos': VideosPhotos    = int(re.findall(r'\d+', l)[0].replace(",",""))

        #Hotel class
        HotelClass = float(str(s.find_all('svg',{'class':'_2aZlo29m'})[0])[17:20]) if len(s.find_all('svg',{'class':'_2aZlo29m'}))==1 else 'null'

        #Count of Number of reviews, Q&As, Room tips
        if len(s.find_all('span',{'class':'_1aRY8Wbl'})) ==0:
            ReviewCount  = 0
            QACount      = 0
            RoomTipCount = 0
        else:
            ReviewCount  = int(s.find_all('span',{'data-test-target':'CC_TAB_Reviews_LABEL'})[0].findChildren('span',{'class':'_1aRY8Wbl'})[0].text.strip().replace(",",""))
            QACount      = int(s.find_all('span',{'data-test-target':'CC_TAB_Questions_LABEL'})[0].findChildren('span',{'class':'_1aRY8Wbl'})[0].text.strip().replace(",",""))
            RoomTipCount = int(s.find_all('span',{'data-test-target':'CC_TAB_RoomTips_LABEL'})[0].findChildren('span',{'class':'_1aRY8Wbl'})[0].text.strip().replace(",",""))

        #Count of Subgroups of Reviews
        ExcellentReviewCount = 0
        VeryGoodReviewCount  = 0
        AverageReviewCount   = 0
        PoorReviewCount      = 0
        TerribleReviewCount  = 0

        if len(s.find_all('div',{'class':'A14zz6ip'})) > 0:
            if int(s.find_all('span',{'class':'_1aRY8Wbl'})[0].text.replace(",",""))>0:
                ExcellentReviewCount = int(s.find_all('span',{'class':'_3fVK8yi6'})[0].text.strip().replace(",",""))
                VeryGoodReviewCount  = int(s.find_all('span',{'class':'_3fVK8yi6'})[1].text.strip().replace(",",""))
                AverageReviewCount   = int(s.find_all('span',{'class':'_3fVK8yi6'})[2].text.strip().replace(",",""))
                PoorReviewCount      = int(s.find_all('span',{'class':'_3fVK8yi6'})[3].text.strip().replace(",",""))
                TerribleReviewCount  = int(s.find_all('span',{'class':'_3fVK8yi6'})[4].text.strip().replace(",",""))

        #Basic information from the About section (Price Range, #Rooms, Location
        PriceRangeLow = 'null'
        PriceRangeHigh= 'null'
        NumberRooms   = 'null'
        Country       = 'null'
        State         = 'null'
        for i in range (len(s.find_all('div',{'class':'_39sLqIkw'}))):
            if s.find_all('div',{'class':'_39sLqIkw'})[i].text == "PRICE RANGE":
                PriceRangeLow = int(re.findall(r'\d+', s.find_all('div',{'class':'_1NHwuRzF'})[i].text.strip())[0].replace(",",""))
                PriceRangeHigh= int(re.findall(r'\d+', s.find_all('div',{'class':'_1NHwuRzF'})[i].text.strip())[1].replace(",",""))
            elif s.find_all('div',{'class':'_39sLqIkw'})[i].text == "NUMBER OF ROOMS":
                NumberRooms   = int(s.find_all('div',{'class':'_1NHwuRzF'})[i].text.strip().replace(",",""))
            elif s.find_all('div',{'class':'_39sLqIkw'})[i].text == "LOCATION":
                listomat=s.find_all('div', {'class':'_1NHwuRzF'})[i].text.split()
                temp=[]
                for e in range(len(listomat)):
                    temp.append(len(re.sub( r"([A-Z])", r" \1",listomat[e]).split()))
                full_split = re.sub( r"([A-Z])", r" \1",s.find_all('div', {'class':'_1NHwuRzF'})[i].text).split()    
                if temp[0]>=3:
                    Country = ''.join(full_split[0:1])
                    State   = ''.join(full_split[1:2])
                elif temp[0]==2:
                    if len(temp)>1:
                        if temp[1]>=2:
                            Country = ''.join(full_split[0:1])
                            State   = ''.join(full_split[1:3])         
                        elif temp[1]==1:
                            if len(temp)>2:
                                if temp[2]>=2:
                                    Country = ''.join(full_split[0:1])
                                    State   = ''.join(full_split[1:4])
                else:
                    if len(temp)>2:
                        if temp[1]>=2:
                            if temp[2]>=2:
                                Country = ''.join(full_split[0:2])
                                State   = ''.join(full_split[2:4])
                            else:
                                Country = ''.join(full_split[0:2])
                                State   = ''.join(full_split[2:5])


        #Languages spoken 
        if len(s.find_all('div',{'class':'_2dtF3ueh'})) > 0:
            LanguagesListed = s.find_all('div',{'class':'_2dtF3ueh'})[-1].text.count(',') +1
            if len(re.findall(r'\d+',s.find_all('div',{'class':'_2dtF3ueh'})[-1].text)) > 0:
                LanguagesAndMore = int(re.findall(r'\d+',s.find_all('div',{'class':'_2dtF3ueh'})[-1].text)[0])
            else:
                LanguagesAndMore = 0
        else:
            LanguagesListed = 0
            LanguagesAndMore= 0
            
        LanguagesSpoken = LanguagesListed + LanguagesAndMore


        #__ out of __ hotels in the region
        NumberInTheRegion='null'
        TotalInTheRegion ='null'
        AccomodationType ='null'
        NameOfTheRegion  ='null'

        if len(s.find_all('div',{'class':'ui_columns _318JyS8B'}))>0:
            if len(s.find_all('div',{'class':'ui_columns _318JyS8B'})[0].findChildren('div',{'class':'ui_column'}))>0:
                if len(s.find_all('div',{'class':'ui_columns _318JyS8B'})[0].findChildren('div',{'class':'ui_column'})[0].findChildren('span',{'class':'_28eYYeHH'}))>0:
                    A = s.find_all('div',{'class':'ui_columns _318JyS8B'})[0].findChildren('div',{'class':'ui_column'})[0].findChildren('span',{'class':'_28eYYeHH'})[0].text.strip().replace(",","")
                    NumberInTheRegion =int(re.findall(r'\d+',A)[0])
                    TotalInTheRegion  =int(re.findall(r'\d+',A)[1])
                    AccomodationType  =A.split()[3].strip()[:-1]
                    if len(s.find_all('div',{'class':'ui_columns _318JyS8B'})[0].findChildren('div',{'class':'ui_column'})[0].findChildren('span',{'class':'_28eYYeHH'})[0].text.strip().split("s in ", 1))>1:
                        NameOfTheRegion = s.find_all('div',{'class':'ui_columns _318JyS8B'})[0].findChildren('div',{'class':'ui_column'})[0].findChildren('span',{'class':'_28eYYeHH'})[0].text.strip().split("s in ", 1)[1].strip()
        
        #BB binary check (if only bb in city)
        BBInnBinary = 'null'
        if len(s.find_all('div',{'id':'taplc_trip_planner_breadcrumbs_0'}))>0:
            BBInnBinary = int('B&B' in s.find_all('div',{'id':'taplc_trip_planner_breadcrumbs_0'})[0].text)


        
        #detailed ratings
        LocationRating    = 'null'
        CleanlinessRating = 'null'
        ServiceRating     = 'null'
        ValueRating       = 'null'
        for i in range (len(s.find_all('div',{'class':'_1krg1t5y'}))):
            if s.find_all('div',{'class':'_1krg1t5y'})[i].findChildren('div',{'class':'_1h7NKZWM'})[0].text == "Location":
                LocationRating = int(re.findall(r'\d+', s.find_all('div',{'class':'_1krg1t5y'})[i].findChildren('span')[0].get('class')[1])[0])
            if s.find_all('div',{'class':'_1krg1t5y'})[i].findChildren('div',{'class':'_1h7NKZWM'})[0].text == "Cleanliness":
                CleanlinessRating = int(re.findall(r'\d+', s.find_all('div',{'class':'_1krg1t5y'})[i].findChildren('span')[0].get('class')[1])[0])    
            if s.find_all('div',{'class':'_1krg1t5y'})[i].findChildren('div',{'class':'_1h7NKZWM'})[0].text == "Service":
                ServiceRating = int(re.findall(r'\d+', s.find_all('div',{'class':'_1krg1t5y'})[i].findChildren('span')[0].get('class')[1])[0])
            if s.find_all('div',{'class':'_1krg1t5y'})[i].findChildren('div',{'class':'_1h7NKZWM'})[0].text == "Value":
                ValueRating = int(re.findall(r'\d+', s.find_all('div',{'class':'_1krg1t5y'})[i].findChildren('span')[0].get('class')[1])[0]) 

        #Room types, ammenities, room features
        CityViewRoomType      = 0
        NonSmokingRoomType    = 0
        SuitesRoomType        = 0
        FamilyRoomType        = 0
        OceanViewRoomType     = 0
        PoolViewRoomType      = 0
        BridalRoomType        = 0

        AmenitiesSectionLength= 0
        AmenitiesBicycleRent  = 0
        AmenitiesHighSpeedWiFi= 0
        AmenitiesRestaurant   = 0
        AmenitiesPaidParking  = 0
        AmenitiesFreeParking  = 0
        AmenitiesFitness      = 0
        AmenitiesBar          = 0
        AmenitiesChildren     = 0
        AmenitiesPets         = 0
        AmenitiesSauna        = 0
        AmenitiesHotTub       = 0
        AmenitiesPool         = 0
        AmenitiesCoffeeShop   = 0
        AmenitiesSpecialDiet  = 0
        AmenitiesBreakfast    = 0
        AmenitiesBreakfastRoom= 0
        AmenitiesTaxi         = 0
        AmenitiesAirport      = 0
        AmenitiesBusiness     = 0
        AmenitiesConference   = 0
        AmenitiesBanquet      = 0
        AmenitiesMeetingRooms = 0
        AmenitiesTerrace      = 0
        AmenitiesBaggage      = 0
        AmenitiesConcierge    = 0
        AmenitiesCurrencyEx   = 0
        AmenitiesNonSmoking   = 0
        Amenities24Hours      = 0
        AmenitiesSpecialCheck = 0
        AmenitiesDryCleaning  = 0
        AmenitiesLaundry      = 0
        AmenitiesFacialTreat  = 0
        AmenitiesSpa          = 0
        AmenitiesMassage      = 0

        RoomFeatSectionLength = 0
        RoomFeatAllergyFree   = 0
        RoomFeatFlatsceenTV   = 0
        RoomFeatWalkInShower  = 0
        RoomFeatHousekeeping  = 0
        RoomFeatCoffeeTea     = 0
        RoomFeatSoundproof    = 0
        RoomFeatSafe          = 0
        RoomFeatLaptopSafe    = 0
        RoomFeatBalcony       = 0
        RoomFeatRefrigerator  = 0
        RoomFeatToiletries    = 0
        RoomFeatHairdryer     = 0
        RoomFeatFireplace     = 0
        RoomFeatAirCon        = 0
        RoomFeatMinibar       = 0
        RoomFeatMicrowave     = 0
        RoomFeatVIP           = 0
        RoomFeatService       = 0
        RoomFeatWakeUp        = 0

        sub = 1 if len(s.find_all('ul',{'class':'_2Buo3Fn4'}))<1 else 0

        for i in range(len(s.find_all('div',{'class':'_1mJdgpMJ'}))):
            if s.find_all('div',{'class':'_1mJdgpMJ'})[i].text == "Room types":
                CityViewRoomType   = int('City view' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                NonSmokingRoomType = int('Non-smoking rooms' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                SuitesRoomType     = int('Suites' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                FamilyRoomType     = int('Family rooms' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                OceanViewRoomType  = int('Ocean view' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                PoolViewRoomType   = int('Pool view' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                BridalRoomType     = int('Bridal suite' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)

            if s.find_all('div',{'class':'_1mJdgpMJ'})[i].text == "Property amenities": 
                AmenitiesSectionLength= len(s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesBicycleRent  = int('Bicycle rental' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesHighSpeedWiFi= int('Free High Speed Internet (WiFi)' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesRestaurant   = int('Restaurant' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesPaidParking  = int('Paid private' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesFreeParking  = int('Free parking' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesFitness      = int('Fitness Center with Gym / Workout Room' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesBar          = int('Bar / lounge' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesChildren     = int('hildren' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesPets         = int('Pets Allowed ( Dog / Pet Friendly )' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesSauna        = int('Sauna' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesHotTub       = int('Hot tub' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesPool         = int('ool' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesCoffeeShop   = int('Coffee shop' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesSpecialDiet  = int('Special diet menus' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesBreakfast    = int('reakfast' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesBreakfastRoom= int('Breakfast in the room' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesTaxi         = int('Taxi service' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesAirport      = int('Airport transportation' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesBusiness     = int('Business Center with Internet Access' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesConference   = int('Conference facilities' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesBanquet      = int('Banquet room' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesMeetingRooms = int('Meeting rooms' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesTerrace      = int('terrace' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesBaggage      = int('Baggage storage' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesConcierge    = int('Concierge' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesCurrencyEx   = int('Currency exchange' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesNonSmoking   = int('Non-smoking hotel' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                Amenities24Hours      = int('24-hour' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesSpecialCheck = int('check-in / check-out' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesDryCleaning  = int('Dry cleaning' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesLaundry      = int('Laundry service' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesFacialTreat  = int('Facial treatments' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesSpa          = int('Spa' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                AmenitiesMassage      = int('assage' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)

            if s.find_all('div',{'class':'_1mJdgpMJ'})[i].text == "Room features": 
                RoomFeatSectionLength= len(s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatAllergyFree  = int('Allergy-free room' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatFlatsceenTV  = int('Flatscreen TV' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatWalkInShower = int('Walk-in shower' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatHousekeeping = int('Housekeeping' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatCoffeeTea    = int('Coffee / tea maker' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatSoundproof   = int('Soundproof rooms' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatSafe         = int('Safe' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatLaptopSafe   = int('Laptop safe' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatBalcony      = int('Private balcony' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatRefrigerator = int('Refrigerator' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatToiletries   = int('Complimentary toiletries' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatHairdryer    = int('Hair dryer' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatFireplace    = int('Fireplace' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatAirCon       = int('Air conditioning' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatMinibar      = int('Minibar' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatMicrowave    = int('Microwave' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatVIP          = int('VIP room facilities' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatService      = int('Room service' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)
                RoomFeatWakeUp       = int('Wake-up service / alarm clock' in s.find_all('div',{'class':'_1nAmDotd'})[i-sub].text)


        #Location scores 
        WalkerScore      = 'null'
        RestaurantScore  = 'null'
        AttractionsScore = 'null'
        if len(s.find_all('span',{'class':'oPMurIUj _1iwDIdby'}))>0:
            WalkerScore = int(s.find_all('span',{'class':'oPMurIUj _1iwDIdby'})[0].text)
        if len(s.find_all('span',{'class':'oPMurIUj TrfXbt7b'}))>0:
            RestaurantScore = int(s.find_all('span',{'class':'oPMurIUj TrfXbt7b'})[0].text)
        if len(s.find_all('span',{'class':'oPMurIUj _1WE0iyL_'}))>0:
            AttractionsScore = int(s.find_all('span',{'class':'oPMurIUj _1WE0iyL_'})[0].text)

        #Desription Stats
        DescriptionLengthWords = 'null'
        DescriptionSustainable = 'null'
        DescriptionGreen       =' null'
        DescriptionRenewables  = 'null'

        if len(s.find_all('div', {'class':'_2f_ruteS _1bona3Pu _2-hMril5 _2uD5bLZZ'}))>0:
            DescriptionLengthWords = len(s.find_all('div', {'class':'_2f_ruteS _1bona3Pu _2-hMril5 _2uD5bLZZ'})[0].text.split())
            DescriptionSustainable = int(any(substring in s.find_all('div', {'class':'_2f_ruteS _1bona3Pu _2-hMril5 _2uD5bLZZ'})[0].text.split() for substring in ["Sustainanability","sustainability","Sustainable","sustainable"]))
            DescriptionGreen       = int(any(substring in s.find_all('div', {'class':'_2f_ruteS _1bona3Pu _2-hMril5 _2uD5bLZZ'})[0].text.split() for substring in ["Green", "green"]))
            DescriptionRenewables  = int(any(substring in s.find_all('div', {'class':'_2f_ruteS _1bona3Pu _2-hMril5 _2uD5bLZZ'})[0].text.split() for substring in ["Renewables","renewables","Renewable","renewable"]))

        #Languages in Reviews
        ReviewLanguages = ['null','null','null','null','null','null','null','null','null','null']
        for i in range(len(s.find_all('span', {'class':'_1wk-I7LS'}))):
            ReviewLanguages[i*2] = s.find_all('span', {'class':'_1wk-I7LS'})[i].text
            ReviewLanguages[i*2+1] = int(re.findall(r'\d+',s.find_all('span', {'class':'mxlinKbW'})[i].text.strip().replace(",",""))[0])


        #HotelType missing
        HotelStyleGreen    = 0
        HotelStyleBusiness = 0
        HotelStyleModern   = 0
        HotelStyleLuxury   = 0

        for m in range(len(s.find_all('div', {'class':'_2dtF3ueh'}))):
            if 'Green' in s.find_all('div', {'class':'_2dtF3ueh'})[m].text:
                HotelStyleGreen    = 1
            if 'Business' in s.find_all('div', {'class':'_2dtF3ueh'})[m].text:
                HotelStyleBusiness = 1
            if 'Modern' in s.find_all('div', {'class':'_2dtF3ueh'})[m].text:
                HotelStyleModern   = 1
            if 'Luxury' in s.find_all('div', {'class':'_2dtF3ueh'})[m].text:
                HotelStyleLuxury   = 1

        #print(HotelListIndexing+counter)
        final_list.append([HotelName, Country, State, HotelClass, AvgRating, PriceRangeLow, PriceRangeHigh,
                           NumberInTheRegion, TotalInTheRegion, AccomodationType, BBInnBinary, NameOfTheRegion, NumberRooms, 
                           LanguagesSpoken, TravelersChoice, GreenLeader, TotalPhotos, DiningPhotos , PoolBeachPhotos, 
                           RoomSuitePhotos, TravelerPhotos,VideosPhotos, ReviewCount, ExcellentReviewCount, 
                           VeryGoodReviewCount, AverageReviewCount, PoorReviewCount, TerribleReviewCount, 
                           QACount, RoomTipCount, LocationRating, CleanlinessRating, ServiceRating, ValueRating,
                           WalkerScore, RestaurantScore, AttractionsScore, DescriptionLengthWords, 
                           DescriptionSustainable, DescriptionGreen, DescriptionRenewables,  ReviewLanguages[0], 
                           ReviewLanguages[1], ReviewLanguages[2], ReviewLanguages[3], ReviewLanguages[4], 
                           ReviewLanguages[5], ReviewLanguages[6], ReviewLanguages[7], ReviewLanguages[8], 
                           ReviewLanguages[9], HotelStyleGreen, HotelStyleBusiness, HotelStyleModern, HotelStyleLuxury,
                           CityViewRoomType, NonSmokingRoomType, SuitesRoomType, FamilyRoomType, OceanViewRoomType, 
                           PoolViewRoomType, BridalRoomType, AmenitiesSectionLength, AmenitiesBicycleRent, 
                           AmenitiesHighSpeedWiFi, AmenitiesRestaurant, AmenitiesPaidParking, AmenitiesFreeParking, 
                           AmenitiesFitness, AmenitiesBar, AmenitiesChildren, AmenitiesPets, AmenitiesSauna, 
                           AmenitiesHotTub, AmenitiesPool, AmenitiesCoffeeShop, AmenitiesSpecialDiet, AmenitiesBreakfast,
                           AmenitiesBreakfastRoom, AmenitiesTaxi, AmenitiesAirport, AmenitiesBusiness, AmenitiesConference, 
                           AmenitiesBanquet, AmenitiesMeetingRooms, AmenitiesTerrace, AmenitiesBaggage, AmenitiesConcierge,
                           AmenitiesCurrencyEx, AmenitiesNonSmoking, Amenities24Hours, AmenitiesSpecialCheck, 
                           AmenitiesDryCleaning, AmenitiesLaundry, AmenitiesFacialTreat, AmenitiesSpa, AmenitiesMassage, 
                           RoomFeatSectionLength, RoomFeatAllergyFree, RoomFeatFlatsceenTV, RoomFeatWalkInShower, 
                           RoomFeatHousekeeping, RoomFeatCoffeeTea, RoomFeatSoundproof, RoomFeatSafe, RoomFeatLaptopSafe,
                           RoomFeatBalcony, RoomFeatRefrigerator, RoomFeatToiletries, RoomFeatHairdryer, RoomFeatFireplace,
                           RoomFeatAirCon, RoomFeatMinibar, RoomFeatMicrowave, RoomFeatVIP, RoomFeatService, RoomFeatWakeUp])
        print(HotelListIndexing+counter)
        counter+=1
        
    final_df = pd.DataFrame(final_list)
    final_df.columns=['HotelName', 'Country', 'State', 'HotelClass', 'AvgRating', 'PriceRangeLow', 'PriceRangeHigh',
                           'NumberInTheRegion', 'TotalInTheRegion', 'AccomodationType', 'BBInnBinary', 'NameOfTheRegion', 'NumberRooms', 
                           'LanguagesSpoken', 'TravelersChoice', 'GreenLeader', 'TotalPhotos', 'DiningPhotos' , 'PoolBeachPhotos', 
                           'RoomSuitePhotos', 'TravelerPhotos','VideosPhotos', 'ReviewCount', 'ExcellentReviewCount', 
                           'VeryGoodReviewCount', 'AverageReviewCount', 'PoorReviewCount', 'TerribleReviewCount', 
                           'QACount', 'RoomTipCount', 'LocationRating', 'CleanlinessRating', 'ServiceRating', 'ValueRating',
                           'WalkerScore', 'RestaurantScore', 'AttractionsScore', 'DescriptionLengthWords', 
                           'DescriptionSustainable', 'DescriptionGreen', 'DescriptionRenewables',  'AllReviewLanguages', 
                           'AllReviewLanguagesCount', 'FirstReviewLanguage', 'FirstReviewLanguageCount', 'SecondReviewLanguage', 
                           'SecondReviewLanguageCount', 'ThirdReviewLanguage', 'ThirdReviewLanguageCount', 'FourthReviewLanguage', 
                           'FourthReviewLanguageCount', 'HotelStyleGreen', 'HotelStyleBusiness', 'HotelStyleModern', 'HotelStyleLuxury',
                           'CityViewRoomType', 'NonSmokingRoomType', 'SuitesRoomType', 'FamilyRoomType', 'OceanViewRoomType', 
                           'PoolViewRoomType', 'BridalRoomType', 'AmenitiesSectionLength', 'AmenitiesBicycleRent', 
                           'AmenitiesHighSpeedWiFi', 'AmenitiesRestaurant', 'AmenitiesPaidParking', 'AmenitiesFreeParking', 
                           'AmenitiesFitness', 'AmenitiesBar', 'AmenitiesChildren', 'AmenitiesPets', 'AmenitiesSauna', 
                           'AmenitiesHotTub', 'AmenitiesPool', 'AmenitiesCoffeeShop', 'AmenitiesSpecialDiet', 'AmenitiesBreakfast',
                           'AmenitiesBreakfastRoom', 'AmenitiesTaxi', 'AmenitiesAirport', 'AmenitiesBusiness', 'AmenitiesConference', 
                           'AmenitiesBanquet', 'AmenitiesMeetingRooms', 'AmenitiesTerrace', 'AmenitiesBaggage', 'AmenitiesConcierge',
                           'AmenitiesCurrencyEx', 'AmenitiesNonSmoking', 'Amenities24Hours', 'AmenitiesSpecialCheck', 
                           'AmenitiesDryCleaning', 'AmenitiesLaundry', 'AmenitiesFacialTreat', 'AmenitiesSpa', 'AmenitiesMassage', 
                           'RoomFeatSectionLength', 'RoomFeatAllergyFree', 'RoomFeatFlatsceenTV', 'RoomFeatWalkInShower', 
                           'RoomFeatHousekeeping', 'RoomFeatCoffeeTea', 'RoomFeatSoundproof', 'RoomFeatSafe', 'RoomFeatLaptopSafe',
                           'RoomFeatBalcony', 'RoomFeatRefrigerator', 'RoomFeatToiletries', 'RoomFeatHairdryer', 'RoomFeatFireplace',
                           'RoomFeatAirCon', 'RoomFeatMinibar', 'RoomFeatMicrowave', 'RoomFeatVIP', 'RoomFeatService', 'RoomFeatWakeUp']
    
    final_df.to_csv(f"FinalDataSet{HotelListIndexing}to{HotelListIndexing+counter-1}.csv",sep=",")
        
    HotelListIndexing+=counter
    